In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [4]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000126                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.000007
0.000750      2.0  0.000134                  2.0 -0.000111
0.001052      3.0  0.000150                  3.0  0.000094
0.001476      4.0  0.000167                  4.0  0.000109
0.002070      5.0  0.000185                  5.0  0.000123
0.002904      6.0  0.000204                  6.0  0.000136
0.004074      7.0  0.000223                  7.0  0.000149
0.005714      8.0  0.000242                  8.0  0.000163
0.008015      9.0  0.000263                  9.0  0.000177
0.011243     10.0  0.000284                 10.0  0.000192
0.015771     11.0  0.000306                 11.0  0.000208
0.022122     12.0  0.000328                 12.0  0.000224
0.031031     13.0  0.000352                 13.0  0.000238
0.043528     14.0  0.000383                 14.0  0.000237
0.061057     15.0  0.000502                 15.0  0.000319
0.085645     16.0  0.000795                 16.0  0.000541
0.120136     17.0  0.001255                 17.0  0.000868
0.168516     18.0  0.001877                 18.0  0.001300
0.236378     19.0  0.002702                 19.0  0.001871
0.331549     20.0  0.003778                 20.0  0.002614
0.465100     21.0  0.005166                 21.0  0.003596
0.652400     22.0  0.006738                 22.0  0.004777
0.915100     23.0  0.007284                 23.0  0.005239
1.283650     24.0  0.006592                 24.0  0.004817
1.800600     25.0  0.005536                 25.0  0.004119
2.525700     26.0  0.004332                 26.0  0.003294
3.542800     27.0  0.003297                 27.0  0.002563
4.969550     28.0  0.002480                 28.0  0.001969
6.970850     29.0  0.001842                 29.0  0.001487
9.778100     30.0  0.001349                 30.0  0.001100
13.715850    31.0  0.000969                 31.0  0.000789
19.239350    32.0  0.000675                 32.0  0.000542
26.987250    33.0  0.000446                 33.0  0.000346
37.855300    34.0  0.000260                 34.0  0.000188
53.100050    35.0  0.000069                 35.0  0.000036
73.887500    36.0 -0.000084                 36.0 -0.000087
97.662500    37.0 -0.000144                 37.0 -0.000138
121.437500   38.0 -0.000118                 38.0 -0.000119
145.212500   39.0 -0.000010                 39.0 -0.000035
168.987500   40.0  0.000163                 40.0  0.000098
192.762500   41.0  0.000571                 41.0  0.000418
216.537500   42.0  0.002126                 42.0  0.001723
240.312500   43.0  0.005712                 43.0  0.005083
264.087500   44.0  0.012184                 44.0  0.011740
287.862500   45.0  0.021898                 45.0  0.022139
311.637500   46.0  0.034684                 46.0  0.035361
335.412500   47.0  0.050020                 47.0  0.050276
359.187500   48.0  0.066521                 48.0  0.067279
382.962500   49.0  0.080351                 49.0  0.082521
406.737500   50.0  0.096366                 50.0  0.097683
430.512500   51.0  0.114525                 51.0  0.111365
454.287500   52.0  0.134803                 52.0  0.125903
478.062500   53.0  0.157161                 53.0  0.144399
501.837500   54.0  0.181221                 54.0  0.167932
525.612500   55.0  0.207107                 55.0  0.195104
549.387500   56.0  0.234232                 56.0  0.222545
573.162500   57.0  0.262301                 57.0  0.247861
596.937500   58.0  0.291114                 58.0  0.271225
620.712500   59.0  0.320073                 59.0  0.294456
644.487500   60.0  0.349238                 60.0  0.321087
668.262500   61.0  0.377849                 61.0  0.352386
692.037500   62.0  0.403856                 62.0  0.385705
715.812500   63.0  0.425231                 63.0  0.416815
739.587500   64.0  0.466503                 64.0  0.468072
76

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -39.984950  0.000000e+00 -39.984950        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -40.085227   0.000183   
6.244000e-04 2     -39.984950  4.137529e-08 -39.984950 -40.085227   0.000183   
8.759000e-04 3     -39.984950  5.826523e-08 -39.984950 -40.085227   0.000183   
1.228600e-03 4     -39.984950  8.261300e-08 -39.984950 -40.085227   0.000183   
1.723400e-03 5     -39.984951  1.177405e-07 -39.984950 -40.085227   0.000183   
2.417400e-03 6     -39.984951  1.684073e-07 -39.984950 -40.085227   0.000183   
3.390900e-03 7     -39.984951  2.414862e-07 -39.984950 -40.085227   0.000183   
4.756500e-03 8     -39.984951  3.468836e-07 -39.984950 -40.085227   0.000183   
6.672000e-03 9     -39.984951  4.988721e-07 -39.984950 -40.085227   0.000184   
9.358900e-03 10    -39.984951  7.180364e-07 -39.984950 -40.085227   0.000184   
1.312780e-02 11    -39.984951  1.034040e-06 -39.984950 -40.085227   0.000184   
1.841450e-02 12    -39.984951  1.489645e-06 -39.984950 -40.085227   0.000184   
2.583020e-02 13    -39.984952  2.146479e-06 -39.984950 -40.085228   0.000185   
3.623230e-02 14    -39.984952  3.093448e-06 -39.984949 -40.085228   0.000185   
5.082340e-02 15    -39.984953  4.468899e-06 -39.984948 -40.085229   0.000186   
7.129060e-02 16    -39.984954  6.646114e-06 -39.984947 -40.085229   0.000188   
1.000000e-01 17    -39.984955  1.050947e-05 -39.984945 -40.085230   0.000191   
1.402710e-01 18    -39.984956  1.760401e-05 -39.984939 -40.085231   0.000196   
1.967600e-01 19    -39.984957  3.056297e-05 -39.984926 -40.085232   0.000205   
2.759970e-01 20    -39.984955  5.405465e-05 -39.984901 -40.085230   0.000221   
3.871000e-01 21    -39.984947  9.630041e-05 -39.984851 -40.085225   0.000251   
5.431000e-01 22    -39.984927  1.717278e-04 -39.984755 -40.085212   0.000303   
7.617000e-01 23    -39.984883  3.017303e-04 -39.984581 -40.085181   0.000396   
1.068500e+00 24    -39.984810  4.937324e-04 -39.984316 -40.085131   0.000537   
1.498800e+00 25    -39.984717  7.371507e-04 -39.983980 -40.085070   0.000721   
2.102400e+00 26    -39.984611  1.026345e-03 -39.983584 -40.085002   0.000948   
2.949000e+00 27    -39.984500  1.350475e-03 -39.983150 -40.084938   0.001215   
4.136600e+00 28    -39.984393  1.707029e-03 -39.982686 -40.084885   0.001522   
5.802500e+00 29    -39.984295  2.098453e-03 -39.982197 -40.084851   0.001876   
8.139200e+00 30    -39.984216  2.529164e-03 -39.981687 -40.084846   0.002283   
1.141700e+01 31    -39.984169  3.006261e-03 -39.981163 -40.084885   0.002749   
1.601470e+01 32    -39.984175  3.539964e-03 -39.980635 -40.084987   0.003281   
2.246400e+01 33    -39.984263  4.143536e-03 -39.980120 -40.085178   0.003887   
3.151050e+01 34    -39.984475  4.832173e-03 -39.979642 -40.085493   0.004572   
4.420010e+01 35    -39.984868  5.615968e-03 -39.979252 -40.085976   0.005338   
6.200000e+01 36    -39.985535  6.430029e-03 -39.979105 -40.086698   0.006136   
8.577500e+01 37    -39.986507  7.164011e-03 -39.979343 -40.087683   0.006875   
1.095500e+02 38    -39.987505  7.755381e-03 -39.979749 -40.088665   0.007469   
1.333250e+02 39    -39.988580  8.498757e-03 -39.980081 -40.089709   0.008179   
1.571000e+02 40    -39.989779  9.669928e-03 -39.980110 -40.090867   0.009237   
1.808750e+02 41    -39.991194  1.154248e-02 -39.979651 -40.092228   0.010875   
2.046500e+02 42    -39.993332  1.528853e-02 -39.978044 -40.094314   0.014137   
2.284250e+02 43    -39.998184  2.612860e-02 -39.972055 -40.099163   0.023838   
2.522000e+02 44    -40.007636  5.166858e-02 -39.955968 -40.108768   0.047759   
2.759750e+02 45    -40.024050  1.023950e-01 -39.921655 -40.125572   0.097630   
2.997500e+02 46    -40.050132  1.901477e-01 -39.859984 -40.152337   0.186753   
3.235250e+02 47    -40.088

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')